In [1]:
import os
import glob
import geopandas as gpd
import pandas as pd
import ast
import json
from shapely.geometry import Point, LineString

import dask.dataframe as dd

In [5]:
csv_file = r'F:\Safegraph\Weekly Places Patterns (for data from 2020-11-30 to Present)\patterns\2021\04\21\09\patterns-part2.csv'

csv_file = r'F:\Safegraph\Monthly Places Patterns (aka Patterns) Jan 2018 - Apr 2020\2020\04\patterns-part4.csv.gz'

csv_file = r'G:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part6.csv.gz'

In [15]:
# find the place-key or safegraph_id in Core_poi

# 2021-01-06, have both place-key and safegraph_place_id
# 2021-03-06, have both place-key and safegraph_place_id
# 2021-04-08, have both place-key and safegraph_place_id
# 2021-05-03, have both place-key and safegraph_place_id
# 2021-06-05, have both place-key and safegraph_place_id
# 2021-07-07, have both place-key ONLY
# 2021-08-06, have both place-key ONLY

csv_file = r'G:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\06\05\00\core_poi-part1.csv.gz'
df1 =  pd.read_csv(csv_file)
df1.head(1)

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,...,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,222-222@5s9-b69-kcq,sg:0024c2de777b43abab827cb0898f3810,NaN,NaN,Copper Cup,NaN,NaN,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515.0,...,MI,49938,US,1.906286e+10,NaN,"Smoothie & Juice Bar,Coffee Shop,Bagel Shop",NaN,NaN,NaN,2019-07


In [16]:
df1.columns

Index(['placekey', 'safegraph_place_id', 'parent_placekey',
       'parent_safegraph_place_id', 'location_name', 'safegraph_brand_ids',
       'brands', 'top_category', 'sub_category', 'naics_code', 'latitude',
       'longitude', 'street_address', 'city', 'region', 'postal_code',
       'iso_country_code', 'phone_number', 'open_hours', 'category_tags',
       'opened_on', 'closed_on', 'tracking_opened_since',
       'tracking_closed_since'],
      dtype='object')

In [2]:
csv_dir = r'F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03'
csv_files = glob.glob(os.path.join(csv_dir, "*.gz"))
csv_files

['F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part1.csv.gz',
 'F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part2.csv.gz',
 'F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part3.csv.gz',
 'F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part4.csv.gz',
 'F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part5.csv.gz',
 'F:\\Safegraph\\Core Places US (Nov 2020 - Present)\\core_poi\\2021\\05\\05\\03\\core_poi-part6.csv.gz']

In [3]:
dfs = []

for f in csv_files[:]:
    print(f)
    df1 =  pd.read_csv(f)
    dfs.append(df1)
    
    print("len(df1):", len(df1))
    
df_all = pd.concat(dfs, axis=0)
df_all

F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part1.csv.gz
1077652
F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part2.csv.gz
1084050
F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part3.csv.gz
1082678
F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part4.csv.gz
1082151
F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part5.csv.gz
1069326
F:\Safegraph\Core Places US (Nov 2020 - Present)\core_poi\2021\05\05\03\core_poi-part6.csv.gz
1022273


,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,...,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
0,227-222@5pw-3bs-p7q,sg:0026d587c9b145b58272cbac7bdea56a,NaN,NaN,Vallone A John DDS,NaN,NaN,Offices of Dentists,Offices of Dentists,621210.0,...,IL,60914,US,1.815939e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1,222-222@8f3-4ts-pvz,sg:0041bdf50f984826acd68ce19922ecd4,NaN,NaN,Sullivan Chris M Dr,NaN,NaN,"Other Professional, Scientific, and Technical ...",Veterinary Services,541940.0,...,LA,71295,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
2,zzy-222@5z5-t79-ty9,sg:005761d9681947629bf02a64cfd4aec2,NaN,NaN,U Pick Carlsbad Strawberry Co,NaN,NaN,Other Miscellaneous Store Retailers,All Other Miscellaneous Store Retailers (excep...,453998.0,...,CA,92008,US,NaN,"{ ""Mon"": [[""9:00"", ""18:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07
3,zzw-222@3x5-6m7-wrk,sg:0066813b8e3b4385adcd9d75f76a2d72,NaN,NaN,Park Heights Baptist Church,NaN,NaN,Religious Organizations,Religious Organizations,813110.0,...,WA,99223,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07
4,226-222@627-w6f-389,sg:007236f9d77d4158bbd4d55f39d98e49,NaN,NaN,CJ's Auto Glass Repair,NaN,NaN,Automotive Repair and Maintenance,Automotive Glass Replacement Shops,811122.0,...,NJ,8816,US,1.732994e+10,"{ ""Mon"": [[""8:00"", ""20:00""]], ""Tue"": [[""8:00"",...",NaN,NaN,NaN,NaN,2019-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022268,224-222@8fw-75p-8y9,sg:ff9f891b41664472bf3fbdab63c4e54c,NaN,NaN,VIP Salon Doral,NaN,NaN,Personal Care Services,Beauty Salons,812112.0,...,FL,33172,US,1.305401e+10,"{ ""Mon"": [], ""Tue"": [[""10:00"", ""19:00""]], ""Wed...",NaN,NaN,NaN,NaN,2019-07
1022269,zzw-222@5nz-kke-wx5,sg:ffb4ed97ab8349b9a82d74e81193a2f0,NaN,NaN,Earling Standard Service,NaN,NaN,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,...,IA,51530,US,1.712747e+10,NaN,NaN,NaN,NaN,NaN,2019-07
1022270,224-223@5x4-4fy-jjv,sg:ffb80fe62c464d99ab73155ed0ada4ee,NaN,NaN,GE Monogram Appliance Repair Service,NaN,NaN,Personal and Household Goods Repair and Mainte...,Appliance Repair and Maintenance,811412.0,...,WA,98119,US,1.425358e+10,"{ ""Mon"": [[""8:30"", ""20:30""]], ""Tue"": [[""8:30"",...",NaN,NaN,NaN,NaN,2019-07
1022271,223-222@63j-r6r-j5f,sg:ffc093f2b41e465f84f7774a88275e1f,NaN,NaN,Speedway,SG_BRAND_93b3d23529ebed78cdf4bfe2e58f1979,Speedway,Gasoline Stations,Gasoline Stations with Convenience Stores,447110.0,...,OH,43551,US,1.419874e+10,NaN,NaN,NaN,NaN,2020-01,2019-07


In [4]:
df_all.dtypes

placekey                      object
safegraph_place_id            object
parent_placekey               object
parent_safegraph_place_id     object
location_name                 object
safegraph_brand_ids           object
brands                        object
top_category                  object
sub_category                  object
naics_code                   float64
latitude                     float64
longitude                    float64
street_address                object
city                          object
region                        object
postal_code                    int64
iso_country_code              object
phone_number                 float64
open_hours                    object
category_tags                 object
opened_on                     object
closed_on                     object
tracking_opened_since         object
tracking_closed_since         object
dtype: object

In [26]:
df_all[['location_name', 'top_category', 'naics_code', 'sub_category', 'category_tags', 'tracking_opened_since', 'tracking_closed_since']].sample(10)

,location_name,top_category,naics_code,sub_category,category_tags,tracking_opened_since,tracking_closed_since
1062515,Foster Corporation,"Sporting Goods, Hobby, and Musical Instrument ...",451120.0,"Hobby, Toy, and Game Stores",NaN,NaN,2019-07
896541,Frank Ellis Photographer,"Other Professional, Scientific, and Technical ...",541921.0,"Photography Studios, Portrait",NaN,NaN,2019-07
493968,North Shore Hawaiian Shave Ice,Restaurants and Other Eating Places,722515.0,Snack and Nonalcoholic Beverage Bars,Ice Cream Shop,NaN,2019-07
362665,Baidoun Fadi MD PC,Offices of Physicians,621111.0,Offices of Physicians (except Mental Health Sp...,NaN,NaN,2019-07
285011,Jos. A. Bank Clothiers,Clothing Stores,448110.0,Men's Clothing Stores,NaN,2019-12,2019-12
82460,Kidds Place Cdc Dunn,Child Day Care Services,624410.0,Child Day Care Services,NaN,NaN,2019-07
54158,Starbucks,Restaurants and Other Eating Places,722515.0,Snack and Nonalcoholic Beverage Bars,"Snacks,Counter Service,Dessert,Tea House,Coffe...",2021-03,2019-07
1039660,Eddie Bauer Outlet,Clothing Stores,448140.0,Family Clothing Stores,NaN,2021-04,2021-04
824356,Subway,Restaurants and Other Eating Places,722513.0,Limited-Service Restaurants,"Counter Service,Sandwich Shop,Lunch,Breakfast,...",NaN,2019-07
584910,Ahava Vida Dental Studio,Offices of Dentists,621210.0,Offices of Dentists,NaN,NaN,2019-07


In [43]:
df_all.groupby(["sub_category"]).count().reindex()


,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,naics_code,latitude,...,region,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since
sub_category,,,,,,,,,,,,,,,,,,,,,
Advertising Agencies,64,64,7,7,64,64,64,64,64,64,...,64,64,64,56,2,0,0,2,64,64
Aircraft Manufacturing,16,16,5,5,16,0,0,16,16,16,...,16,16,16,0,0,0,0,0,0,16
All Other Amusement and Recreation Industries,25368,25368,3361,3361,25368,1125,1125,25368,25368,25368,...,25368,25368,25368,18034,10511,1,46,2157,791,25295
All Other Automotive Repair and Maintenance,3460,3460,303,303,3460,2508,2508,3460,3460,3460,...,3460,3460,3460,2661,2376,0,10,114,1022,3300
All Other Business Support Services,112,112,3,3,112,111,111,112,112,112,...,112,112,112,88,6,0,0,0,0,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wind Electric Power Generation,663,663,8,8,663,0,0,663,663,663,...,663,663,663,0,0,0,0,0,0,663
Wineries,5973,5973,222,222,5973,0,0,5973,5973,5973,...,5973,5973,5973,5141,3783,5937,0,454,0,5973
Wireless Telecommunications Carriers (except Satellite),50472,50472,13951,13951,50472,31973,31973,50472,50472,50472,...,50472,50472,50472,40316,23996,0,4906,8427,21532,46020


In [12]:
regions = df_all['region'].unique()
regions = list(regions)
regions

['FL',
 'OK',
 'VA',
 'NV',
 'GA',
 'AL',
 'CO',
 'CA',
 'MA',
 'PA',
 'AZ',
 'NE',
 'NC',
 'MI',
 'VT',
 'OH',
 'NM',
 'NY',
 'MO',
 'MT',
 'KY',
 'NJ',
 'IL',
 'TX',
 'MN',
 'IA',
 'IN',
 'WA',
 'TN',
 'CT',
 'MD',
 'AK',
 'OR',
 'SC',
 'WY',
 'LA',
 'NH',
 'AR',
 'ID',
 'MS',
 'WI',
 'ME',
 'ND',
 'DE',
 'UT',
 'SD',
 'DC',
 'RI',
 'KS',
 'HI',
 'WV',
 'PR',
 'GU',
 'VI',
 'AS',
 'MP']

In [13]:
for region in regions:
    re_df = df_all[df_all['region'] == region]
    gdf = gpd.GeoDataFrame(re_df, geometry=gpd.points_from_xy(re_df.longitude, re_df.latitude))
    gdf.to_file(f'POI202104_{region}.shp')
    print(f"{region} done.")
print("All done.")

C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  after removing the cwd from sys.path.


FL done.
OK done.
VA done.
NV done.
GA done.
AL done.
CO done.
CA done.
MA done.
PA done.
AZ done.
NE done.
NC done.
MI done.
VT done.
OH done.
NM done.
NY done.
MO done.
MT done.
KY done.
NJ done.
IL done.
TX done.
MN done.
IA done.
IN done.
WA done.
TN done.
CT done.
MD done.
AK done.
OR done.
SC done.
WY done.
LA done.
NH done.
AR done.
ID done.
MS done.
WI done.
ME done.
ND done.
DE done.
UT done.
SD done.
DC done.
RI done.
KS done.
HI done.
WV done.
PR done.
GU done.
VI done.
AS done.
MP done.
All done.


In [5]:
gdf = gpd.GeoDataFrame(df_all, geometry=gpd.points_from_xy(df_all.longitude, df_all.latitude))

NameError: name 'df_all' is not defined

In [25]:
gdf

,placekey,safegraph_place_id,parent_placekey,parent_safegraph_place_id,location_name,safegraph_brand_ids,brands,top_category,sub_category,naics_code,...,postal_code,iso_country_code,phone_number,open_hours,category_tags,opened_on,closed_on,tracking_opened_since,tracking_closed_since,geometry
0,223-222@627-w3m-nt9,sg:0006fc2aeb06472d8d47f576494c1ae9,zzy-222@627-w3m-nwk,sg:5b6704a373af42fd8c6be80c49d78356,Five Guys,SG_BRAND_87b52f7d201822f23c1015f8e9beb4b1,Five Guys,Restaurants and Other Eating Places,Limited-Service Restaurants,722513.0,...,7080,US,1.908756e+10,NaN,"Burgers,Counter Service,Lunch,Dinner",NaN,NaN,NaN,2019-07,POINT (-74.42561 40.54959)
1,222-222@5pr-6kg-435,sg:00092a86c412461f9c888c5f54d93b34,NaN,NaN,Funcoland,NaN,NaN,Consumer Goods Rental,Video Tape and Disc Rental,532282.0,...,64138,US,1.816768e+10,NaN,NaN,NaN,NaN,NaN,2019-07,POINT (-94.51893 38.95887)
2,226-224@5vg-85k-789,sg:00215697429b4d93be3159e936080e5a,NaN,NaN,Farmstead Cheeses and Wines,NaN,NaN,"Beer, Wine, and Liquor Stores","Beer, Wine, and Liquor Stores",445310.0,...,94501,US,NaN,"{ ""Mon"": [[""9:00"", ""20:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07,POINT (-122.23945 37.76777)
3,222-222@5pt-wkn-w6k,sg:0022a49e24244ebb8717abd145a70f14,NaN,NaN,Cuts N More,NaN,NaN,Personal Care Services,Other Personal Care Services,812199.0,...,47993,US,1.765763e+10,NaN,NaN,NaN,NaN,NaN,2019-07,POINT (-87.30394 40.29771)
4,222-223@8g7-83t-73q,sg:00568635a45f442fb0879fb0fc668ca7,222-224@8g7-83t-73q,sg:bb99bc86fea840bca915b1aa1186a6d5,Circle K,SG_BRAND_4cbffa4318cebdb09227afca3fe16846,Circle K,Grocery Stores,Convenience Stores,445120.0,...,30809,US,NaN,"{ ""Mon"": [[""0:00"", ""24:00""]], ""Tue"": [[""0:00"",...",NaN,NaN,NaN,NaN,2019-07,POINT (-82.12577 33.53241)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1049169,zzz-222@5xc-jvq-gtv,sg:ff6f7c7e3e1b4ac9876261b221fe7f83,NaN,NaN,Columbia Park,NaN,NaN,"Museums, Historical Sites, and Similar Institu...",Nature Parks and Other Similar Institutions,712190.0,...,97203,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07,POINT (-122.71078 45.58021)
1049170,zzy-222@8f5-sfb-jy9,sg:ff83a2ea750f4bcfa0368badbbab9d5f,NaN,NaN,Spring Hill Cemetery,NaN,NaN,Death Care Services,Cemeteries and Crematories,812220.0,...,38916,US,NaN,NaN,NaN,NaN,NaN,NaN,2019-07,POINT (-89.29646 33.74473)
1049171,zzw-222@5q3-3zp-f9f,sg:ff9b9bb5da934091bc4bf46a5dd0b33d,NaN,NaN,Community Connections,NaN,NaN,Individual and Family Services,Services for the Elderly and Persons with Disa...,624120.0,...,50219,US,1.641628e+10,NaN,NaN,NaN,NaN,NaN,2019-07,POINT (-92.92142 41.47428)
1049172,zzy-222@bp4-7rc-59f,sg:ffdca919604b42a8a0c4993d3ec98fa0,NaN,NaN,Hamakua Mushrooms,NaN,NaN,Grocery Stores,Supermarkets and Other Grocery (except Conveni...,445110.0,...,96764,US,1.808962e+10,"{ ""Mon"": [[""9:00"", ""16:00""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,NaN,2019-07,POINT (-155.23577 19.98479)


In [26]:
gdf.to_file(os.path.join(csv_dir, "POI_202104.shp"))

C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [23]:
df = pd.read_csv(csv_file)
df.dtypes

placekey                      object
safegraph_place_id            object
parent_placekey               object
parent_safegraph_place_id     object
location_name                 object
safegraph_brand_ids           object
brands                        object
top_category                  object
sub_category                  object
naics_code                   float64
latitude                     float64
longitude                    float64
street_address                object
city                          object
region                        object
postal_code                    int64
iso_country_code              object
phone_number                 float64
open_hours                    object
category_tags                 object
opened_on                     object
closed_on                     object
tracking_opened_since         object
tracking_closed_since         object
dtype: object

In [12]:
columns = list(df.columns)
for c in columns:
    len_c = ''
    cell = df.iloc[0][c]
    if str(cell)[0] == "[":
        a_list = ast.literal_eval(cell)
        len_c = len(a_list)
    if str(cell)[0] == "{":
        jdata = json.loads(cell)

        len_c = len(jdata.keys())
    print(f"{c}:, len: {len_c}     \n  {cell}\n")

placekey:, len:      
  223-222@627-w3m-nt9

safegraph_place_id:, len:      
  sg:0006fc2aeb06472d8d47f576494c1ae9

parent_placekey:, len:      
  zzy-222@627-w3m-nwk

parent_safegraph_place_id:, len:      
  sg:5b6704a373af42fd8c6be80c49d78356

location_name:, len:      
  Five Guys

safegraph_brand_ids:, len:      
  SG_BRAND_87b52f7d201822f23c1015f8e9beb4b1

brands:, len:      
  Five Guys

top_category:, len:      
  Restaurants and Other Eating Places

sub_category:, len:      
  Limited-Service Restaurants

naics_code:, len:      
  722513.0

latitude:, len:      
  40.549593

longitude:, len:      
  -74.425611

street_address:, len:      
  5020 Hadley Center Dr

city:, len:      
  South Plainfield

region:, len:      
  NJ

postal_code:, len:      
  7080

iso_country_code:, len:      
  US

phone_number:, len:      
  19087563334.0

open_hours:, len:      
  nan

category_tags:, len:      
  Burgers,Counter Service,Lunch,Dinner

opened_on:, len:      
  nan

closed_on:, len:

# Geometry footprint

In [3]:
Geo_2020_8_file = r'H:\Safegraph\Geometry Footprint\August2020Release\SafeGraphPlacesGeoSupplementSquareFeet.csv.gz'
Geo_2020_7_file = r'H:\Safegraph\Geometry Footprint\July2020Release\SafeGraphPlacesGeoSupplementSquareFeet.csv.gz'
Geo_2020_5_file = r'H:\Safegraph\Geometry Footprint\May2020Release\SafeGraphPlacesGeoSupplementSquareFeet.csv.gz'

df_2020_5 = pd.read_csv(Geo_2020_5_file)
df_2020_7 = pd.read_csv(Geo_2020_7_file)
df_2020_8 = pd.read_csv(Geo_2020_8_file)

df_all = pd.concat([df_2020_5, df_2020_7, df_2020_8])

no_duplicate_id_all_df = df_all.drop_duplicates(subset=['safegraph_place_id'])

no_duplicate_id_all_df.to_csv(r'H:\Safegraph\Geometry Footprint\Geo_footprint_unqiue.csv', index=False)

no_duplicate_id_all_df

In [4]:
df_2020_5 = pd.read_csv(Geo_2020_5_file)
df_2020_7 = pd.read_csv(Geo_2020_7_file)
df_2020_8 = pd.read_csv(Geo_2020_8_file)

In [5]:
print("len(df_2020_5), len(df_2020_7), len(df_2020_8):", len(df_2020_5), len(df_2020_7), len(df_2020_8))

len(df_2020_5), len(df_2020_7), len(df_2020_8): 5929331 6093918 5304326


In [6]:
df_all = pd.concat([df_2020_5, df_2020_7, df_2020_8])

df_all

,safegraph_place_id,location_name,polygon_class,is_synthetic,includes_parking_lot,iso_country_code,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,Liberty Tax Service,SHARED_POLYGON,False,False,CA,38193
1,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,OWNED_POLYGON,False,False,US,5495
2,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,OWNED_POLYGON,False,NaN,US,388046
3,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,SHARED_POLYGON,False,False,US,687
4,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,SHARED_POLYGON,False,False,US,10925
...,...,...,...,...,...,...,...
5304321,sg:ffc8dcd3067f402ebd253c37f8bdc408,Steve's Auto Service and Repair,OWNED_POLYGON,False,False,US,7795
5304322,sg:ffdb0820a4544f7e8a2956473b378f3e,Remington College Dallas Campus,SHARED_POLYGON,False,True,US,140429
5304323,sg:ffde4d09a5e741eaad2efdc3c8e2ef5a,Caliber Collision,OWNED_POLYGON,False,False,US,22808
5304324,sg:ffeeec97d26841f1b0bfbd709a0438bb,Divers Inn,SHARED_POLYGON,False,False,US,3246


In [8]:
no_duplicate_row_all_df = df_all.drop_duplicates()

In [9]:
no_duplicate_row_all_df

,safegraph_place_id,location_name,polygon_class,is_synthetic,includes_parking_lot,iso_country_code,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,Liberty Tax Service,SHARED_POLYGON,False,False,CA,38193
1,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,OWNED_POLYGON,False,False,US,5495
2,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,OWNED_POLYGON,False,NaN,US,388046
3,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,SHARED_POLYGON,False,False,US,687
4,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,SHARED_POLYGON,False,False,US,10925
...,...,...,...,...,...,...,...
5304198,sg:f65d6f34aeaa43bc8343527d1a90544c,Stop &amp Shop Pharmacy,SHARED_POLYGON,False,False,US,73067
5304258,sg:fae0371cbbbd42fab5f9200e31e67b7d,Gaithersburg Pharmacy,OWNED_POLYGON,False,False,US,3246
5304275,sg:fc4e6104556a45a2aa5a0d6cdf8bc44e,U S Renal Care Chula Vista Broadway Dialysis,SHARED_POLYGON,False,False,US,12644
5304278,sg:fc776bc4545f4df8b7db01b0f373346f,Western Union,SHARED_POLYGON,False,False,US,2843


In [10]:
no_duplicate_id_all_df = df_all.drop_duplicates(subset=['safegraph_place_id'])
no_duplicate_id_all_df

,safegraph_place_id,location_name,polygon_class,is_synthetic,includes_parking_lot,iso_country_code,area_square_feet
0,sg:000190647c574858930fbf286b4e8ab8,Liberty Tax Service,SHARED_POLYGON,False,False,CA,38193
1,sg:001341fe7e794ab6bd65bb80759a1ac6,Old Firehouse School,OWNED_POLYGON,False,False,US,5495
2,sg:002921847f104c93a6df47bb831dedd2,Tenth Avenue Park,OWNED_POLYGON,False,NaN,US,388046
3,sg:002d2e9f047d47ca8661abb8939096cf,Hunter Village Wines & Liquors,SHARED_POLYGON,False,False,US,687
4,sg:0060616e5f184585a026a0b7e8192fbd,Fat Moe's,SHARED_POLYGON,False,False,US,10925
...,...,...,...,...,...,...,...
5304108,sg:ed977afc64994ab4a09ade0854eb26d2,Western Union,SHARED_POLYGON,False,False,US,15057
5304198,sg:f65d6f34aeaa43bc8343527d1a90544c,Stop &amp Shop Pharmacy,SHARED_POLYGON,False,False,US,73067
5304258,sg:fae0371cbbbd42fab5f9200e31e67b7d,Gaithersburg Pharmacy,OWNED_POLYGON,False,False,US,3246
5304275,sg:fc4e6104556a45a2aa5a0d6cdf8bc44e,U S Renal Care Chula Vista Broadway Dialysis,SHARED_POLYGON,False,False,US,12644


In [11]:
no_duplicate_id_all_df.to_csv(r'H:\Safegraph\Geometry Footprint\Geo_footprint_unqiue.csv', index=False)

# Geometry

In [3]:
import glob
import os

import pandas as pd

geometry_dir = r'G:\Safegraph\Geometry'
geometry_csvs = glob.glob(os.path.join(geometry_dir, '*.csv'))


dfs = []

for f in geometry_csvs:
    print(f)
    df = pd.read_csv(f)
    dfs.append(df)
    
df_all = pd.concat(dfs)
df_all

G:\Safegraph\Geometry\Part 1.csv
G:\Safegraph\Geometry\Part 2.csv
G:\Safegraph\Geometry\Part 3.csv
G:\Safegraph\Geometry\Part 4.csv
G:\Safegraph\Geometry\Part 5.csv
G:\Safegraph\Geometry\Part 6.csv


,placekey,sg_g.safegraph_place_id,sg_g.parent_safegraph_place_id,sg_g.location_name,sg_g.brands,sg_g.latitude,sg_g.longitude,sg_g.street_address,sg_g.city,sg_g.region,sg_g.postal_code,sg_g.iso_country_code,sg_g.polygon_wkt,sg_g.polygon_class,sg_g.includes_parking_lot,sg_g.is_synthetic,sg_g.building_height,sg_g.enclosed
0,222-222@39z-7y7-6x5,sg:75b3e2c29fd34f7299e0a7737d503a1a,NaN,United States Postal Service (USPS),United States Postal Service (USPS),66.908244,-156.884844,9998 Airport Rd,Kobuk,AK,99751,US,POLYGON ((-156.88441233384538 66.9082459999999...,OWNED_POLYGON,NaN,True,NaN,False
1,222-222@3bh-zxm-vxq,sg:b68063de95464712a3fc947fb2af3a9e,NaN,North Star Metals,NaN,60.498474,-151.009717,42365 Sterling Hwy,Soldotna,AK,99669,US,"POLYGON ((-151.009805617922 60.49841070824056,...",OWNED_POLYGON,False,False,NaN,False
2,222-222@3bk-j6j-h89,sg:e45d5514b3ec463c9928f66f8c074c64,NaN,Angelo's Trattoria,NaN,57.809173,-152.366553,2625 Mill Bay Rd,Kodiak,AK,99615,US,NaN,NaN,NaN,False,NaN,False
3,222-222@3bs-km2-mff,sg:55b7beefac284f0b86150f1c0ae776bd,NaN,My Mom's Thai Cuisine,NaN,62.160438,-150.031865,37180 Talkeetna Spur Rd,Susitna North,AK,99676,US,"POLYGON ((-150.03195 62.160413, -150.03195 62....",OWNED_POLYGON,False,False,NaN,False
4,222-224@3bt-c2c-p35,sg:1b2a121c94154e4e90c6f816d65e44df,sg:e8a1b1b24ad2462c9965a8d2f98aba43,Faith Lutheran Church,NaN,61.173257,-149.839195,5200 Lake Otis Pkwy,Anchorage,AK,99507,US,"POLYGON ((-149.839298 61.173266, -149.83927 61...",SHARED_POLYGON,False,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122267,zzz-222@8ts-3m6-t5f,sg:762454f081d14271a1cf2c93cbabbd7b,NaN,La Madera Park,NaN,32.259503,-110.931304,2700 E La Madera Dr,Tucson,AZ,85716,US,POLYGON ((-110.93172194729692 32.2602851403854...,OWNED_POLYGON,True,False,NaN,False
1122268,zzz-222@8ts-w4f-nt9,sg:e33cd4fb483d4368851ccb6250767ed5,NaN,Four Peaks Park,NaN,33.620630,-111.709564,14825 N del Cambre Ave Fountain Hills Az 85268,Fountain Hills,AZ,85268,US,POLYGON ((-111.70884144157924 33.6203756000539...,OWNED_POLYGON,NaN,False,NaN,False
1122269,zzz-222@8ts-zgw-nyv,sg:1a0a4a37a09f4981a3311625ab0b4b95,sg:12639d3b4427460c8785fbb9b6bb549c,Vista Del Camino Mckellips Lake Park,NaN,33.459251,-111.913902,7700 E Roosevelt St,Scottsdale,AZ,85257,US,MULTIPOLYGON (((-111.91543793781057 33.4582938...,OWNED_POLYGON,NaN,False,NaN,False
1122270,zzz-222@bp5-ckb-66k,sg:b071af617675455f9d9c9d2afa3299df,NaN,Paukukalo Beach,NaN,20.910263,-156.485556,Paukukalo Beach,Wailuku,HI,96793,US,"POLYGON ((-156.48459437607437 20.9102778, -156...",OWNED_POLYGON,NaN,True,NaN,False


In [2]:
import geopandas as gpd
from shapely import wkt

gdf = gpd.GeoDataFrame(df_all, geometry=gpd.GeoSeries.from_wkt(df_all['sg_g.polygon_wkt']))

NameError: name 'df_all' is not defined

In [4]:
import geopandas
from shapely.geometry import Polygon

def load_wkt(row):
    print(row)
    wkts = row['sg_g.polygon_wkt'].to_list()
    
    polygons = []
    for w in wkts:
        polygon = wkt.loads(w)
        polygons.append(polygon)
        
    multipolyton = shapely.gemetry.MultiPolygon(polygons)    
    row['geometry'] = multipolyton
    return multipolyton

def load_wkt2(row):
   # print(row)
    w = row['sg_g.polygon_wkt'] 
    polygon = wkt.loads(w)
    
    #polygon = Polygon(polygon)    
    row['geometry'] = polygon
    return polygon

df_all_cleaned = df_all[~df_all['sg_g.polygon_wkt'].isna()]

del df_all

df_all_cleaned

,placekey,sg_g.safegraph_place_id,sg_g.parent_safegraph_place_id,sg_g.location_name,sg_g.brands,sg_g.latitude,sg_g.longitude,sg_g.street_address,sg_g.city,sg_g.region,sg_g.postal_code,sg_g.iso_country_code,sg_g.polygon_wkt,sg_g.polygon_class,sg_g.includes_parking_lot,sg_g.is_synthetic,sg_g.building_height,sg_g.enclosed
0,222-222@39z-7y7-6x5,sg:75b3e2c29fd34f7299e0a7737d503a1a,NaN,United States Postal Service (USPS),United States Postal Service (USPS),66.908244,-156.884844,9998 Airport Rd,Kobuk,AK,99751,US,POLYGON ((-156.88441233384538 66.9082459999999...,OWNED_POLYGON,NaN,True,NaN,False
1,222-222@3bh-zxm-vxq,sg:b68063de95464712a3fc947fb2af3a9e,NaN,North Star Metals,NaN,60.498474,-151.009717,42365 Sterling Hwy,Soldotna,AK,99669,US,"POLYGON ((-151.009805617922 60.49841070824056,...",OWNED_POLYGON,False,False,NaN,False
3,222-222@3bs-km2-mff,sg:55b7beefac284f0b86150f1c0ae776bd,NaN,My Mom's Thai Cuisine,NaN,62.160438,-150.031865,37180 Talkeetna Spur Rd,Susitna North,AK,99676,US,"POLYGON ((-150.03195 62.160413, -150.03195 62....",OWNED_POLYGON,False,False,NaN,False
4,222-224@3bt-c2c-p35,sg:1b2a121c94154e4e90c6f816d65e44df,sg:e8a1b1b24ad2462c9965a8d2f98aba43,Faith Lutheran Church,NaN,61.173257,-149.839195,5200 Lake Otis Pkwy,Anchorage,AK,99507,US,"POLYGON ((-149.839298 61.173266, -149.83927 61...",SHARED_POLYGON,False,False,NaN,False
5,222-223@3bt-c2c-p35,sg:e8a1b1b24ad2462c9965a8d2f98aba43,NaN,Faith Evangelical Lutheran School,NaN,61.173178,-149.839184,5200 Lake Otis Pkwy,Anchorage,AK,99507,US,POLYGON ((-149.83995701059393 61.1735889080216...,OWNED_POLYGON,True,False,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122267,zzz-222@8ts-3m6-t5f,sg:762454f081d14271a1cf2c93cbabbd7b,NaN,La Madera Park,NaN,32.259503,-110.931304,2700 E La Madera Dr,Tucson,AZ,85716,US,POLYGON ((-110.93172194729692 32.2602851403854...,OWNED_POLYGON,True,False,NaN,False
1122268,zzz-222@8ts-w4f-nt9,sg:e33cd4fb483d4368851ccb6250767ed5,NaN,Four Peaks Park,NaN,33.620630,-111.709564,14825 N del Cambre Ave Fountain Hills Az 85268,Fountain Hills,AZ,85268,US,POLYGON ((-111.70884144157924 33.6203756000539...,OWNED_POLYGON,NaN,False,NaN,False
1122269,zzz-222@8ts-zgw-nyv,sg:1a0a4a37a09f4981a3311625ab0b4b95,sg:12639d3b4427460c8785fbb9b6bb549c,Vista Del Camino Mckellips Lake Park,NaN,33.459251,-111.913902,7700 E Roosevelt St,Scottsdale,AZ,85257,US,MULTIPOLYGON (((-111.91543793781057 33.4582938...,OWNED_POLYGON,NaN,False,NaN,False
1122270,zzz-222@bp5-ckb-66k,sg:b071af617675455f9d9c9d2afa3299df,NaN,Paukukalo Beach,NaN,20.910263,-156.485556,Paukukalo Beach,Wailuku,HI,96793,US,"POLYGON ((-156.48459437607437 20.9102778, -156...",OWNED_POLYGON,NaN,True,NaN,False


In [5]:
df_all_cleaned = df_all_cleaned.reset_index()

In [6]:
geometry = df_all_cleaned.apply(load_wkt2, axis=1)

In [7]:
gdf = gpd.GeoDataFrame(df_all_cleaned, geometry=geometry)

In [9]:
states = gdf['sg_g.region'].unique()
states

array(['AK', 'MT', 'WA', 'ID', 'SD', 'IA', 'NE', 'MO', 'KS', 'MN', 'ND',
       'WI', 'IL', 'KY', 'TN', 'AL', 'MS', 'AR', 'IN', 'CO', 'NM', 'UT',
       'WY', 'OK', 'TX', 'MI', 'OH', 'CA', 'OR', 'NV', 'AZ', 'NY', 'NJ',
       'CT', 'PA', 'MA', 'MD', 'NH', 'ME', 'RI', 'WV', 'VA', 'NC', 'DE',
       'SC', 'VT', 'FL', 'LA', 'GA', 'HI', 'PR', 'DC', 'VI', 'GU', 'MP',
       'AS'], dtype=object)

In [14]:
saved_path = r'K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois'
for idx, state in enumerate(states):
    
    state_gdf = gdf[gdf['sg_g.region'] == state].drop(columns=['sg_g.polygon_wkt'])
    new_name = os.path.join(saved_path, f'{state}_POI.shp')
    print("Processing:", idx, state, new_name, len(state_gdf))
    state_gdf.to_file(new_name)

Processing: 0 AK K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\AK_POI.shp 14395
Processing: 1 MT K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\MT_POI.shp 23917
Processing: 2 WA K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\WA_POI.shp 133333
Processing: 3 ID K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\ID_POI.shp 32200
Processing: 4 SD K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\SD_POI.shp 19967
Processing: 5 IA K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\IA_POI.shp 65673
Processing: 6 NE K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\NE_POI.shp 38192
Processing: 7 MO K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\MO_POI.shp 111429
Proces

In [18]:
gdf[['placekey', 'sg_g.safegraph_place_id']].to_csv(r'K:\OneDrive_USC\OneDrive - University of South Carolina\Research\Mobility_covid\CORE_pois\placeKey_mapping.csv', index=False)

In [17]:
gdf[['placekey', 'sg_g.safegraph_place_id']]

,placekey,sg_g.safegraph_place_id
0,222-222@39z-7y7-6x5,sg:75b3e2c29fd34f7299e0a7737d503a1a
1,222-222@3bh-zxm-vxq,sg:b68063de95464712a3fc947fb2af3a9e
2,222-222@3bs-km2-mff,sg:55b7beefac284f0b86150f1c0ae776bd
3,222-224@3bt-c2c-p35,sg:1b2a121c94154e4e90c6f816d65e44df
4,222-223@3bt-c2c-p35,sg:e8a1b1b24ad2462c9965a8d2f98aba43
...,...,...
5941706,zzz-222@8ts-3m6-t5f,sg:762454f081d14271a1cf2c93cbabbd7b
5941707,zzz-222@8ts-w4f-nt9,sg:e33cd4fb483d4368851ccb6250767ed5
5941708,zzz-222@8ts-zgw-nyv,sg:1a0a4a37a09f4981a3311625ab0b4b95
5941709,zzz-222@bp5-ckb-66k,sg:b071af617675455f9d9c9d2afa3299df
